In [39]:
#Import necessary Libraries
import pandas as pd
import numpy as np
import requests
import urllib
import urllib3
from gzip import io
import gzip
import re
import tqdm
import time

In [722]:
df = pd.read_csv('movie_dataset.csv')
df = df.drop(columns=["Unnamed: 0"])
print(len(df))
df.head()

3923


,actors,characters,movie title,genres,release year,IMDB ID,Scripts
0,Meryl Streep,Clarissa Vaughan,The Hours,Drama,2002,tt0274558,"Dearest, I feel certain that I am going mad ag..."
1,Meryl Streep,Blue Mecha (voice),A.I. Artificial Intelligence,"Adventure, Drama, Science Fiction",2001,tt0212720,Al That was when the icecaps had melted becaus...
2,Meryl Streep,Francesca Johnson,The Bridges of Madison County,"Drama, Romance",1995,tt0112579,"- MICHAEL: Hi, sis. - CAROLYN: Hey. - CAROLYN:..."
3,Meryl Streep,Miranda Priestly,The Devil Wears Prada,"Comedy, Drama, Romance",2006,tt0458352,"Good luck. Hi. Uh, I have an appointment with ..."
4,Meryl Streep,Clara del Valle Trueba,The House of the Spirits,"Drama, Romance",1994,tt0107151,THE HOUSE OF THE SPIRITS. It’s good to be back...


In [725]:
df.tail(50)

,actors,characters,movie title,genres,release year,IMDB ID,Scripts
3873,Will Ferrell,Jack Wyatt,Bewitched,"Comedy, Fantasy, Romance",2005,tt0374536,"It's perfect. Oh, great. I'll take it. Wonderf..."
3874,Will Ferrell,Franz Liebkind,The Producers,Comedy,2005,tt0395251,It's time for Max To put his back ers on their...
3875,Will Ferrell,Chazz Michael Michaels,Blades of Glory,Comedy,2007,tt0445934,I'll take him. Jimmy MacElroy. The crowd swoon...
3876,Will Ferrell,Ron Burgundy,"Wake Up, Ron Burgundy: The Lost Movie",Comedy,2004,tt0423505,NaN
3877,Will Ferrell,Jackie Moon,Semi-Pro,Comedy,2008,tt0839980,"# Baby, who's ready to lick me sexy? # # Baby,..."
3878,Will Ferrell,Detective Allen Gamble,The Other Guys,"Action, Comedy, Crime",2010,tt1386588,Advertise your product or brand here contact w...
3879,Will Ferrell,Phil Weston,Kicking & Screaming,"Comedy, Family",2005,tt0384642,I was born a baby. A blank slate. Thinking I w...
3880,Will Ferrell,Sky Corrigan,Superstar,"Comedy, Family",1999,tt0167427,NaN
3881,Will Ferrell,Corbit,Winter Passing,"Comedy, Drama",2006,tt0380817,[Man] Yeah! [Woman] There are certain hours of...
3882,Will Ferrell,the tenant,The Landlord,NaN,2007,tt1160329,NaN


import pandas as pd
import numpy as np
import urllib
import re
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import requests
import numpy as np
import io
import json
import gzip

In [17]:
len(df['actors'].unique())

102

Unreleased movies are present in the dataframe. Time to take them out 

Actors, movie titles and release years are complete.

Now, our dataset looks somewhat ready for exploratorary analysis.

In [12]:
df.count()

actors          3923
characters      3813
movie title     3923
genres          3808
release year    3923
IMDB ID         3905
Scripts         3923
dtype: int64

In [13]:
pd.value_counts(df['actors']).plot.bar(figsize=(30,15))

In [439]:
def get_movie_text(movie):
    query1 = urllib.parse.quote(movie)
    query2 = '/sublanguageid-eng'
    
    #The structure of the url is https://rest.opensubtitles.org/search/query-<MOVIE_NAME>. 
    #I added the query2 to only return subtitles in english
    dat = requests.get("https://rest.opensubtitles.org/search/imdbid-"+query1+query2,headers={"user-agent":"RosebudA"})
    dat=dat.json()

    #The opensubtitles search is not great so we have to look for a movie name that exactly matches the movie name 
    #and used that as an input. We select the first occurence hence the [0] at the end
    index=[i for i in np.arange(len(dat)) if dat[i]["MatchedBy"]=="imdbid"][1]

    link = dat[index]["SubDownloadLink"] 
    response = urllib.request.urlopen(link)
    compressed_file = io.BytesIO(response.read())
    decompressed_file = gzip.GzipFile(fileobj=compressed_file)

    #Download the file and save using the same movie name
    with open(movie, 'wb') as outfile:
        outfile.write(decompressed_file.read())
    
        #Read the downloaded file
    with open(movie, errors='replace') as f:
        lines = f.readlines()
    '''
    The structure of a subtitle file is ID, time tag and text. IDs are just numbers from 1-n
    time tag specifices the time the text should be shown and is up to milliseconds 
    and is in the format 00:00:00:00 ---> 00:00:00:001 where the time before the arrow is the start time to show the text andthe time after the arrow is the end time'''
          ########CLEANUP#######
    #Some of the text are in html like tags <i></i>, so we remove that. We also remove the newline character \n
    lines=[i.replace("<i>","").replace("</i>","").replace("\n","") for i in lines]
    
     #     #The first and last IDs are usually advertisements so we want to start our text from the second ID and end before the last
    try:    
        beginning=lines.index("2")
        IDs=[i for i in lines if i.isnumeric()]
        lastID=IDs[-1]
        end=lines.index(lastID)
        lines = lines[beginning:end]
    
            #Remove remove traling and leading whitespaces and all empty lines
        lines=[i.strip() for i in lines if i.strip() !=""]
    
        #Now want to extract just the text and leave the timestamps and IDs. There are several ways to do this but I will go for the 
        #safe method of matching the timestamps and IDs then removing them
    
        #The regex here mataches 2 digits with a column seperating them
        timestamps=[i for i in lines if re.match("\d{2}:\d{2}",i)!= None]
        lines=[i for i in lines if i not in timestamps and i not in IDs]
        text=" ".join(lines)
    except ValueError:
        index=[i for i in np.arange(len(dat)) if dat[i]["MatchedBy"]=="imdbid"][3]

        link = dat[index]["SubDownloadLink"] 
        response = urllib.request.urlopen(link)
        compressed_file = io.BytesIO(response.read())
        decompressed_file = gzip.GzipFile(fileobj=compressed_file)

        #Download the file and save using the same movie name
        with open(movie, 'wb') as outfile:
            outfile.write(decompressed_file.read())
    
            #Read the downloaded file
        with open(movie, errors='replace') as f:
            lines = f.readlines()
        '''
        The structure of a subtitle file is ID, time tag and text. IDs are just numbers from 1-n
        time tag specifices the time the text should be shown and is up to milliseconds 
        and is in the format 00:00:00:00 ---> 00:00:00:001 where the time before the arrow is the start time to show the text andthe time after the arrow is the end time'''
              ########CLEANUP#######
        #Some of the text are in html like tags <i></i>, so we remove that. We also remove the newline character \n
        lines=[i.replace("<i>","").replace("</i>","").replace("\n","") for i in lines]
          #  return lines
    
         #     #The first and last IDs are usually advertisements so we want to start our text from the second ID and end before the last
        start=lines.index("2")
        IDs=[i for i in lines if i.isnumeric()]
        lastID=IDs[-1]
        end=lines.index(lastID)
        lines = lines[start:end]
    
                #Remove remove traling and leading whitespaces and all empty lines
        lines=[i.strip() for i in lines if i.strip() !=""]
            
            #Now want to extract just the text and leave the timestamps and IDs. There are several ways to do this but I will go for the 
            #safe method of matching the timestamps and IDs then removing them
            
            #The regex here mataches 2 digits with a column seperating them
        timestamps=[i for i in lines if re.match("\d{2}:\d{2}",i)!= None]
        lines=[i for i in lines if i not in timestamps and i not in IDs]
        text=" ".join(lines)
    except ValueError:
        index=[i for i in np.arange(len(dat)) if dat[i]["MatchedBy"]=="imdbid"][2]

        link = dat[index]["SubDownloadLink"] 
        response = urllib.request.urlopen(link)
        compressed_file = io.BytesIO(response.read())
        decompressed_file = gzip.GzipFile(fileobj=compressed_file)

        #Download the file and save using the same movie name
        with open(movie, 'wb') as outfile:
            outfile.write(decompressed_file.read())
    
            #Read the downloaded file
        with open(movie, errors='replace') as f:
            lines = f.readlines()
        '''
        The structure of a subtitle file is ID, time tag and text. IDs are just numbers from 1-n
        time tag specifices the time the text should be shown and is up to milliseconds 
        and is in the format 00:00:00:00 ---> 00:00:00:001 where the time before the arrow is the start time to show the text andthe time after the arrow is the end time'''
              ########CLEANUP#######
        #Some of the text are in html like tags <i></i>, so we remove that. We also remove the newline character \n
        lines=[i.replace("<i>","").replace("</i>","").replace("\n","") for i in lines]
          #  return lines
    
         #     #The first and last IDs are usually advertisements so we want to start our text from the second ID and end before the last
        start=lines.index("2")
        IDs=[i for i in lines if i.isnumeric()]
        lastID=IDs[-1]
        end=lines.index(lastID)
        lines = lines[start:end]
    
                #Remove remove traling and leading whitespaces and all empty lines
        lines=[i.strip() for i in lines if i.strip() !=""]
            
            #Now want to extract just the text and leave the timestamps and IDs. There are several ways to do this but I will go for the 
            #safe method of matching the timestamps and IDs then removing them
            
            #The regex here mataches 2 digits with a column seperating them
        timestamps=[i for i in lines if re.match("\d{2}:\d{2}",i)!= None]
        lines=[i for i in lines if i not in timestamps and i not in IDs]
        text=" ".join(lines)
    except ValueError:
        index=[i for i in np.arange(len(dat)) if dat[i]["MatchedBy"]=="imdbid"][3]

        link = dat[index]["SubDownloadLink"] 
        response = urllib.request.urlopen(link)
        compressed_file = io.BytesIO(response.read())
        decompressed_file = gzip.GzipFile(fileobj=compressed_file)

        #Download the file and save using the same movie name
        with open(movie, 'wb') as outfile:
            outfile.write(decompressed_file.read())
    
            #Read the downloaded file
        with open(movie, errors='replace') as f:
            lines = f.readlines()
        '''
        The structure of a subtitle file is ID, time tag and text. IDs are just numbers from 1-n
        time tag specifices the time the text should be shown and is up to milliseconds 
        and is in the format 00:00:00:00 ---> 00:00:00:001 where the time before the arrow is the start time to show the text andthe time after the arrow is the end time'''
              ########CLEANUP#######
        #Some of the text are in html like tags <i></i>, so we remove that. We also remove the newline character \n
        lines=[i.replace("<i>","").replace("</i>","").replace("\n","") for i in lines]
          #  return lines
    
         #     #The first and last IDs are usually advertisements so we want to start our text from the second ID and end before the last
        start=lines.index("2")
        IDs=[i for i in lines if i.isnumeric()]
        lastID=IDs[-1]
        end=lines.index(lastID)
        lines = lines[start:end]
    
                #Remove remove traling and leading whitespaces and all empty lines
        lines=[i.strip() for i in lines if i.strip() !=""]
            
            #Now want to extract just the text and leave the timestamps and IDs. There are several ways to do this but I will go for the 
            #safe method of matching the timestamps and IDs then removing them
            
            #The regex here mataches 2 digits with a column seperating them
        timestamps=[i for i in lines if re.match("\d{2}:\d{2}",i)!= None]
        lines=[i for i in lines if i not in timestamps and i not in IDs]
        text=" ".join(lines)
    except ValueError:
        index=[i for i in np.arange(len(dat)) if dat[i]["MatchedBy"]=="imdbid"][4]

        link = dat[index]["SubDownloadLink"] 
        response = urllib.request.urlopen(link)
        compressed_file = io.BytesIO(response.read())
        decompressed_file = gzip.GzipFile(fileobj=compressed_file)

        #Download the file and save using the same movie name
        with open(movie, 'wb') as outfile:
            outfile.write(decompressed_file.read())
    
            #Read the downloaded file
        with open(movie, errors='replace') as f:
            lines = f.readlines()
        '''
        The structure of a subtitle file is ID, time tag and text. IDs are just numbers from 1-n
        time tag specifices the time the text should be shown and is up to milliseconds 
        and is in the format 00:00:00:00 ---> 00:00:00:001 where the time before the arrow is the start time to show the text andthe time after the arrow is the end time'''
              ########CLEANUP#######
        #Some of the text are in html like tags <i></i>, so we remove that. We also remove the newline character \n
        lines=[i.replace("<i>","").replace("</i>","").replace("\n","") for i in lines]
          #  return lines
    
         #     #The first and last IDs are usually advertisements so we want to start our text from the second ID and end before the last
        start=lines.index("2")
        IDs=[i for i in lines if i.isnumeric()]
        lastID=IDs[-1]
        end=lines.index(lastID)
        lines = lines[start:end]
    
                #Remove remove traling and leading whitespaces and all empty lines
        lines=[i.strip() for i in lines if i.strip() !=""]
            
            #Now want to extract just the text and leave the timestamps and IDs. There are several ways to do this but I will go for the 
            #safe method of matching the timestamps and IDs then removing them
            
            #The regex here mataches 2 digits with a column seperating them
        timestamps=[i for i in lines if re.match("\d{2}:\d{2}",i)!= None]
        lines=[i for i in lines if i not in timestamps and i not in IDs]
        text=" ".join(lines)
        
        
    return text

In [689]:
query1 = urllib.parse.quote('tt0892782')
query2 = '/sublanguageid-eng'
    
#The structure of the url is https://rest.opensubtitles.org/search/query-<MOVIE_NAME>. 
#I added the query2 to only return subtitles in english
dat = requests.get("https://rest.opensubtitles.org/search/imdbid-"+query1+query2,headers={"user-agent":"kingsleytorlowei"})
dat=dat.json()

#The opensubtitles search is not great so we have to look for a movie name that exactly matches the movie name 
#and used that as an input. We select the first occurence hence the [0] at the end
index=[i for i in np.arange(len(dat)) if dat[i]["MatchedBy"]=="imdbid"][0]

link = dat[index]["SubDownloadLink"] 
response = urllib.request.urlopen(link)
compressed_file = io.BytesIO(response.read())
decompressed_file = gzip.GzipFile(fileobj=compressed_file)

#Download the file and save using the same movie name
with open('tt0892782', 'wb') as outfile:
    outfile.write(decompressed_file.read())
    
    #Read the downloaded file
with open('tt0892782', errors='replace') as f:
    lines = f.readlines()
'''
The structure of a subtitle file is ID, time tag and text. IDs are just numbers from 1-n
time tag specifices the time the text should be shown and is up to milliseconds 
and is in the format 00:00:00:00 ---> 00:00:00:001 where the time before the arrow is the start time to show the text andthe time after the arrow is the end time'''
      ########CLEANUP#######
#Some of the text are in html like tags <i></i>, so we remove that. We also remove the newline character \n
lines=[i.replace("<i>","").replace("</i>","").replace("\n","") for i in lines]
  #  return lines
    
 #     #The first and last IDs are usually advertisements so we want to start our text from the second ID and end before the last
start=lines.index("2")
IDs=[i for i in lines if i.isnumeric()]
lastID=IDs[-1]
end=lines.index(lastID)
lines = lines[start:end]
    
        #Remove remove traling and leading whitespaces and all empty lines
lines=[i.strip() for i in lines if i.strip() !=""]
    
    #Now want to extract just the text and leave the timestamps and IDs. There are several ways to do this but I will go for the 
    #safe method of matching the timestamps and IDs then removing them
    
    #The regex here mataches 2 digits with a column seperating them
timestamps=[i for i in lines if re.match("\d{2}:\d{2}",i)!= None]
lines=[i for i in lines if i not in timestamps and i not in IDs]
text=" ".join(lines)

In [688]:
#movie_ids = list(df['IMDB ID'])
movie_ids[3814:]

['tt0892782',
 'tt1155056',
 'tt0427152',
 'tt0120772',
 'tt1045778',
 nan,
 'tt1655460',
 'tt1659337',
 'tt0111033',
 'tt1758830',
 'tt1229340',
 'tt2195548',
 'tt1814621',
 nan,
 'tt1462901',
 'tt2398249',
 'tt1754656',
 'tt3498820',
 'tt0458364',
 'tt0478970',
 'tt5574372',
 'tt1700841',
 'tt0124315',
 'tt1637706',
 'tt0499554',
 'tt0477347',
 'tt2452386',
 'tt5829040',
 'tt0137338',
 'tt0401244',
 'tt0469897',
 'tt0380609',
 'tt0210719',
 'tt0320668',
 'tt0251094',
 'tt0119557',
 'tt0283288',
 'tt0358526',
 'tt6358260',
 'tt0107592',
 'tt0253425',
 'tt0117660',
 'tt0413356',
 'tt0259210',
 'tt3404006',
 'tt0118655',
 'tt0420223',
 'tt0261392',
 'tt0319343',
 'tt0302886',
 'tt0357413',
 'tt0838283',
 'tt0373445',
 'tt0335438',
 'tt0196229',
 'tt0120770',
 'tt0396269',
 'tt0378947',
 'tt0415306',
 'tt0374536',
 'tt0395251',
 'tt0445934',
 'tt0423505',
 'tt0839980',
 'tt1386588',
 'tt0384642',
 'tt0167427',
 'tt0380817',
 'tt1160329',
 'tt1001526',
 'tt1386011',
 'tt1531663',
 'tt0144

In [712]:
scripts = []
for i in (movie_ids[3881:]):
    try:
        scripts.append(get_movie_text(i))
    except IndexError:
   #     continue
#    else:
        scripts.append('')

In [713]:
len(scripts)

42

In [714]:
for i in scripts:
    list_of_scripts.append(i)

In [715]:
len(list_of_scripts)

3923

In [719]:
df.head()

,actors,characters,movie title,genres,release year,IMDB ID
0,Meryl Streep,Clarissa Vaughan,The Hours,Drama,2002,tt0274558
1,Meryl Streep,Blue Mecha (voice),A.I. Artificial Intelligence,"Adventure, Drama, Science Fiction",2001,tt0212720
2,Meryl Streep,Francesca Johnson,The Bridges of Madison County,"Drama, Romance",1995,tt0112579
3,Meryl Streep,Miranda Priestly,The Devil Wears Prada,"Comedy, Drama, Romance",2006,tt0458352
4,Meryl Streep,Clara del Valle Trueba,The House of the Spirits,"Drama, Romance",1994,tt0107151


In [720]:
script = pd.Series(list_of_scripts)
df['Scripts'] = script.values
df.head()

,actors,characters,movie title,genres,release year,IMDB ID,Scripts
0,Meryl Streep,Clarissa Vaughan,The Hours,Drama,2002,tt0274558,"Dearest, I feel certain that I am going mad ag..."
1,Meryl Streep,Blue Mecha (voice),A.I. Artificial Intelligence,"Adventure, Drama, Science Fiction",2001,tt0212720,Al That was when the icecaps had melted becaus...
2,Meryl Streep,Francesca Johnson,The Bridges of Madison County,"Drama, Romance",1995,tt0112579,"- MICHAEL: Hi, sis. - CAROLYN: Hey. - CAROLYN:..."
3,Meryl Streep,Miranda Priestly,The Devil Wears Prada,"Comedy, Drama, Romance",2006,tt0458352,"Good luck. Hi. Uh, I have an appointment with ..."
4,Meryl Streep,Clara del Valle Trueba,The House of the Spirits,"Drama, Romance",1994,tt0107151,THE HOUSE OF THE SPIRITS. It’s good to be back...


In [726]:
df.to_csv(path_or_buf='moviedataset.csv')

In [690]:
list_of_scripts.append(text)

In [707]:
list_of_scripts.append('')

In [696]:
list_of_scripts.pop()

''

In [425]:
#List of missing subtitles
#53, 67, 111, 114, 120, 131, 137, 170, 168, 172, 176, 177, 181, 215, 217

In [1128]:
empty_script= []
for script in list_of_scripts:
    if len(script) < 4:
        empty_script.append(script)

In [1129]:
len(empty_script)

123